In [36]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


Import Log

In [37]:
log = xes_importer.apply("noisy_log.xes")

parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 9583.28it/s]


In [38]:

bpmn_graph = pm4py.read_bpmn("../diagram.bpmn")
net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

Find Deviations

In [39]:
found_allignments = alignments.apply(log, net, im, fm)

aligning log, completed variants :: 100%|██████████| 20/20 [00:00<00:00, 459.44it/s]


In [40]:
found_allignments

[{'alignment': [('A_submitted', 'A_submitted'),
   ('A_partlysubmitted', 'A_partlysubmitted'),
   ('A_preaccepted', 'A_preaccepted'),
   ('A_accepted', 'A_accepted'),
   ('A_finalized', 'A_finalized'),
   ('A_approved', 'A_approved'),
   ('A_registered', 'A_registered'),
   ('A_activated', 'A_activated'),
   ('>>', None),
   ('>>', None)],
  'cost': 2,
  'visited_states': 10,
  'queued_states': 31,
  'traversed_arcs': 31,
  'lp_solved': 4,
  'fitness': 1.0,
  'bwc': 110004},
 {'alignment': [('A_submitted', 'A_submitted'),
   ('A_partlysubmitted', 'A_partlysubmitted'),
   ('A_preaccepted', 'A_preaccepted'),
   ('A_accepted', 'A_accepted'),
   ('A_finalized', 'A_finalized'),
   ('A_approved', 'A_approved'),
   ('A_activated', 'A_activated'),
   ('A_registered', 'A_registered'),
   ('>>', None),
   ('>>', None)],
  'cost': 2,
  'visited_states': 10,
  'queued_states': 31,
  'traversed_arcs': 31,
  'lp_solved': 5,
  'fitness': 1.0,
  'bwc': 110004},
 {'alignment': [('A_submitted', 'A_submi

In [54]:
import pandas as pd
trace_attribute_values = [trace.attributes['Bank'] for trace in log]
attribute_count = {}
for attribute in trace_attribute_values:
    if attribute in attribute_count:
        attribute_count[attribute] += 1
    else:
        attribute_count[attribute] = 1
attributes = sorted(attribute_count.keys())
deviations = ['Deviation', 'No Deviation']
lables = attributes + deviations
sankey_value = {}

for i, found_allignment in enumerate(found_allignments):
    
    if found_allignment['cost'] > 1000:
        if log[i].attributes['Bank'] not  in sankey_value:
            sankey_value[log[i].attributes['Bank']] = [0,1]
        else:
            sankey_value[log[i].attributes['Bank']] [1] += 1
    else:
        if log[i].attributes['Bank'] not  in sankey_value:
            sankey_value[log[i].attributes['Bank']]  = [1,0]
        else:
            sankey_value[log[i].attributes['Bank']][0] += 1
source = []
target = []
value = []
for key in sankey_value:
    positions = lables.index(key)
    source.append(positions)
    source.append(positions)
    target.append(3)
    target.append(4)
    value.append(sankey_value[key][0])
    value.append(sankey_value[key][1])
    

[2, 2, 0, 0, 1, 1]
[3, 4, 3, 4, 3, 4]
[253, 0, 265, 234, 248, 0]


In [64]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = lables,
      color = "blue"
    ),
    link = dict(
      source = source, 
      target = target,
      value = value
  ))])

fig.update_layout(title_text="Deviations based on Trace Attribute", font_size=10)
fig.update_layout(width=int(400))
fig.show()
